In [ ]:
import os, sys, glob

# adds the package path to the Python path to make sure all the local imports work fine 
if os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))) not in sys.path:
    sys.path.append(os.path.dirname(os.path.dirname(os.path.dirname(os.getcwd()))))

# local imports 
from wp4.constants import POLLUTANTS, DATA_DIR_CAMS, DB_HOST, DB_NAME, DB_USER, DB_PASS, DATA_DIR_PLOTS

# import remaining packages needed for the script
import xarray as xr
import pandas as pd
import psycopg2 
from datetime import datetime, timedelta
import math
from pathlib import Path

In [ ]:
def to_datetime(x):
    """Convert the date into a datetime object"""
    date = x['DATE']
    hour = x['TIME']
    
    if hour is "":
        return dt.datetime.strptime(x['DATE'], "%Y-%m-%d")
    
    datetime_str = f'{date} {hour}'
    
    if len(hour) == 4:
        return dt.datetime.strptime(datetime_str, "%Y-%m-%d %H%M")
    else:
        return dt.datetime.strptime(datetime_str, "%Y-%m-%d %H:%M:00")    

In [ ]:
# some functions to calculate a bounding box for a given lat long coordinate, found on stackoverflow: 
# https://stackoverflow.com/questions/238260/how-to-calculate-the-bounding-box-for-a-given-lat-lng-location

# degrees to radians
def deg2rad(degrees):
    return math.pi*degrees/180.0
# radians to degrees
def rad2deg(radians):
    return 180.0*radians/math.pi

# Semi-axes of WGS-84 geoidal reference
WGS84_a = 6378137.0  # Major semiaxis [m]
WGS84_b = 6356752.3  # Minor semiaxis [m]

# Earth radius at a given latitude, according to the WGS-84 ellipsoid [m]
def WGS84EarthRadius(lat):
    # http://en.wikipedia.org/wiki/Earth_radius
    An = WGS84_a*WGS84_a * math.cos(lat)
    Bn = WGS84_b*WGS84_b * math.sin(lat)
    Ad = WGS84_a * math.cos(lat)
    Bd = WGS84_b * math.sin(lat)
    return math.sqrt( (An*An + Bn*Bn)/(Ad*Ad + Bd*Bd) )

# Bounding box surrounding the point at given coordinates,
# assuming local approximation of Earth surface as a sphere
# of radius given by WGS84
def boundingBox(latitudeInDegrees, longitudeInDegrees, halfSideInKm):
    lat = deg2rad(latitudeInDegrees)
    lon = deg2rad(longitudeInDegrees)
    halfSide = 1000*halfSideInKm

    # Radius of Earth at given latitude
    radius = WGS84EarthRadius(lat)
    # Radius of the parallel at given latitude
    pradius = radius*math.cos(lat)

    latMin = lat - halfSide/radius
    latMax = lat + halfSide/radius
    lonMin = lon - halfSide/pradius
    lonMax = lon + halfSide/pradius

    return (rad2deg(latMin), rad2deg(lonMin), rad2deg(latMax), rad2deg(lonMax))

In [ ]:
ds = xr.open_dataset(f"{DATA_DIR_CAMS}/o3_conc.nc").copy()
ds['fire_mask'] = ds[list(ds.data_vars.keys())[0]] > 0
    
# select fire events for to mask
conn = psycopg2.connect(dbname=DB_NAME, user=DB_USER, password=DB_PASS, host=DB_HOST)
    
query = f"""
    SELECT id, datetime, ST_X(geometry), ST_Y(geometry), source, location, reference, type, info
    FROM public.all_fire_events
    WHERE reference = 'Aqua' OR reference = 'Terra'
"""

df = pd.read_sql_query(query,con=conn).rename(columns = {'st_x':'longitude', 'st_y':'latitude'})
    
conn.close()

for ind, fe in df.iterrows():
    date = fe['datetime']

    # create a bounding box of 100 by 100 km
    bb = boundingBox(fe['latitude'], fe['longitude'], 50) 
    # select min and max longitude and latitude to select the cams data within the bounding box
    min_lon = 360 + bb[1]
    min_lat = bb[0]
    max_lon = 360 + bb[3]
    max_lat = bb[2]
    
    min_time= date.replace(hour=0, minute=0)- timedelta(hours=12)
    max_time= date.replace(hour=0, minute=0)+ timedelta(hours=12)
    
    mask_lon = (ds.longitude >= min_lon) & (ds.longitude <= max_lon)
    mask_lat = (ds.latitude >= min_lat) & (ds.latitude <= max_lat)
    mask_time = (ds.time.to_pandas() >= min_time).to_xarray() & (ds.time.to_pandas() <= max_time).to_xarray()
    
    ds['fire_mask'] = ds['fire_mask'].where(cond=~(mask_lon & mask_lat & mask_time), other=False)

In [ ]:
ds['fire_mask'].to_netcdf(Path(DATA_DIR_CAMS).joinpath('fire_mask.nc'))